In [1]:
a = (1, )
len(a)

1

In [2]:
a = ([1,2,23], )
len(a)

1

In [5]:
import pandas as pd
import sqlite3
from sqlite3 import Error
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [31]:
# Creating our connection with the non_normalized database
conn = create_connection('non_normalized.db')

# Fetching all the exams in a list
sql_statement = "SELECT DISTINCT Exams from Students ORDER BY Exams"
exams = execute_sql_statement(sql_statement, conn)
exams = list(map(lambda row: (row[0]), exams))

# Creating a hashmap for exam and year where key is exam and value is the year 
hasho = {}
for exam in exams:
    if("," not in exam):
        key, value = exam.split(" ")
        if key in hasho:
            continue
        else:
            hasho[key] = int(value.replace("(", "").replace(")", ""))
    else:
        list_of_exams = exam.split(",")
        for exam in list_of_exams:
            key, value = exam.strip().split()
            if key not in hasho:
                hasho[key] = int(value.replace("(", "").replace(")", ""))
            else:
                continue             
hasho   
    
        
        
    


{'exam1': 2016,
 'exam10': 2019,
 'exam9': 2018,
 'exam7': 2017,
 'exam4': 2019,
 'exam8': 2018,
 'exam6': 2017,
 'exam5': 2020,
 'exam3': 2018,
 'exam2': 2017}

In [14]:
for exam in exams:
    if("," not in exam):
        continue
    

hi


In [24]:
list_of_exams = exams[1].split(",")
list_of_exams

for exam in list_of_exams:
    print(exam.strip().split(" "))

['exam1', '(2016)']
['exam10', '(2019)']
['exam9', '(2018)']
['exam1', '(2016)']
['exam9', '(2018)']
['exam7', '(2017)']
['exam1', '(2016)']


In [28]:
hasho = {}
for exam in exams:
    if("," not in exam):
        key, value = exam.split(" ")
        if key in hasho:
            continue
        else:
            hasho[key] = value
    else:
        list_of_exams = exam.split(",")
        for exam in list_of_exams:
            key, value = exam.strip().split()
            if key not in hasho:
                hasho[key] = value
            else:
                continue             
hasho   
    
        
        
    

{'exam1': '(2016)',
 'exam10': '(2019)',
 'exam9': '(2018)',
 'exam7': '(2017)',
 'exam4': '(2019)',
 'exam8': '(2018)',
 'exam6': '(2017)',
 'exam5': '(2020)',
 'exam3': '(2018)',
 'exam2': '(2017)'}

In [35]:
list_of_tuples_exams_years = []
for exam, year in hasho.items():
    list_of_tuples_exams_years.append((exam, year))
list_of_tuples_exams_years
    


[('exam1', 2016),
 ('exam10', 2019),
 ('exam9', 2018),
 ('exam7', 2017),
 ('exam4', 2019),
 ('exam8', 2018),
 ('exam6', 2017),
 ('exam5', 2020),
 ('exam3', 2018),
 ('exam2', 2017)]

In [42]:
for exam_and_year in list_of_tuples_exams_years:
    print((exam_and_year[0], exam_and_year[1]))
    print("________")
    

('exam1', 2016)
________
('exam10', 2019)
________
('exam9', 2018)
________
('exam7', 2017)
________
('exam4', 2019)
________
('exam8', 2018)
________
('exam6', 2017)
________
('exam5', 2020)
________
('exam3', 2018)
________
('exam2', 2017)
________


In [39]:
type(list_of_tuples_exams_years[0][1])

int

exam1 2016
-
exam10 2019
-
exam9 2018
-
exam7 2017
-
exam4 2019
-
exam8 2018
-
exam6 2017
-
exam5 2020
-
exam3 2018
-
exam2 2017
-


In [38]:
hasho

{'exam1': 2016,
 'exam10': 2019,
 'exam9': 2018,
 'exam7': 2017,
 'exam4': 2019,
 'exam8': 2018,
 'exam6': 2017,
 'exam5': 2020,
 'exam3': 2018,
 'exam2': 2017}

In [ ]:
# Creating query to make new Exams Table
create_table_sql = """CREATE TABLE [Exams] (
[Exam] NOT NULL PRIMARY KEY,
[Year] INTEGER NOT NULL 
);
"""
# Setting up connection with the normalized db 
conn_norm = create_connection('normalized.db')

# Creating our Exams table 
create_table(conn_norm, create_table_sql)
# conn_norm.close()


def insert_exams(conn, values):
    sql = ''' INSERT INTO Exams(Exams)
            VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

conn_norm = create_connection('normalized.db')
with conn_norm:
    for exam_and_year in list_of_tuples_exams_years:
        insert_exams(conn_norm, exam_and_year)